In [0]:
from pyspark.sql import functions as F

silver = spark.table("workspace.pessoas.silver_pessoas")

gold = (silver
  .withColumn("nome_completo", F.concat_ws(" ", F.col("primeiro_nome"), F.col("ultimo_nome")))
  .withColumn("email_dominio", F.element_at(F.split(F.col("email"), "@"), 2))
  .withColumn("idade", F.floor(F.months_between(F.current_date(), F.col("nascimento"))/12))
  .withColumn("faixa_etaria",
              F.when(F.col("idade") < 18, "Menor de 18")
               .when(F.col("idade") <= 24, "18-24")
               .when(F.col("idade") <= 34, "25-34")
               .when(F.col("idade") <= 44, "35-44")
               .when(F.col("idade") <= 54, "45-54")
               .when(F.col("idade") <= 64, "55-64")
               .otherwise("65+"))
  .withColumn("faixa_ordem",
              F.when(F.col("faixa_etaria") == "Menor de 18", 0)
               .when(F.col("faixa_etaria") == "18-24", 1)
               .when(F.col("faixa_etaria") == "25-34", 2)
               .when(F.col("faixa_etaria") == "35-44", 3)
               .when(F.col("faixa_etaria") == "45-54", 4)
               .when(F.col("faixa_etaria") == "55-64", 5)
               .otherwise(6))
  .withColumn("ano_nascimento", F.year(F.col("nascimento")))
)

spark.sql("DROP TABLE IF EXISTS workspace.pessoas.gold_pessoas")
gold.write.mode("overwrite").format("delta").saveAsTable("workspace.pessoas.gold_pessoas")

spark.sql("SELECT * FROM workspace.pessoas.gold_pessoas LIMIT 10").show()
